In [ ]:
import numpy as np
import pandas as pd
import random

from collections import defaultdict
import pickle
from scipy.sparse import diags, vstack, hstack, lil_matrix, csr_matrix
import networkx as nx
import argparse
import numpy as np, scipy.stats as st


dictionary_name_cat = {'abortion':{'R' : 'Pro-life',
                        'B' : 'Pro-choice'},
                       'guns':{'R' : 'Control',
                        'B' : 'Rights'},
                       'evolution':{'R' : 'Creationism',
                        'B' : 'Evol. Bio.'},
                       'lgbt':{'R':'Discrimination',
                              'B':'Support'},
                       'racism':{'R':'Racism ',
                                'B':'Anti-racism'},
                       'cannabis':{'R':'Prohibiiton',
                                  'B': 'Activism'}
                      }

def average_opposite_color(from_color, to_color, A, alpha=0.2, iterations=15):
    
    initial_red = np.array([1 for n in from_color])
    norm_initial_red = initial_red/np.sum(initial_red)
    p = np.zeros((1, A.shape[0]))
    p[:,from_color] = norm_initial_red


    # Confirmation bias
    #conf_bias = np.zeros((1, A.shape[0]))
    #conf_bias[:,from_color] = conf_bias[:,from_color] + conf_bias_param*conf_bias[:,from_color]

    #iterations = 15
    p_new = p @ A
    p_new = p_new/np.sum(p_new)
    d = [1/(i+1) for i in p_new[0]]
    diagonal = diags(d, offsets=0)
    D = (diagonal @ A.transpose()).transpose()
    norm = [1/i[0] if i!=0 else 1 for i in np.array(D.sum(axis=1))[:]]
    D = diags(norm, offsets=0) @ D
    #rrrr = [np.mean(p_new[:,to_color][p_new[:,to_color]!=0])]
    #mmmm = [np.mean(p_new[:,to_color])]
    ssss = [np.sum(p_new[:,to_color])]
    same = [np.sum(p_new[:,from_color])]
    #same_avg = [np.mean(p_new[:,from_color])]
    #same_avg_ = [np.mean(p_new[:,from_color][p_new[:,from_color]!=0])]
    #nnnn = [np.sum(p_new[:,to_color])/(np.sum(p_new[:,to_color])+np.sum(p_new[:,from_color]))]
    
    return ssss, same#mmmm#rrrr, mmmm, ssss, same, same_avg, same_avg_, nnnn

def average_opposite_color_bootstrapping(from_color, to_color, A, alpha=0.2, iterations=15):
    

    p_unif = 1/len(from_color)

    initial_red = np.array([1 for n in from_color])
    norm_initial_red = initial_red/np.sum(initial_red)
    p = np.zeros((1, A.shape[0]))
    indices = from_color

    to_mod = []
    upd_indices = []
    for i in indices:
        if i not in to_mod:
            to_mod += [i]
        else:
            upd_indices += [i]

    initial_red = np.array(list(set(from_color)))
    p[:,initial_red] = [p_unif]*len(initial_red)

    #print(np.sum(p))

    while len(to_mod) > 0:
        indices = upd_indices
        to_mod = []
        upd_indices = []
        #print(indices)
        for i in indices:
            if i not in to_mod:
                to_mod += [i]
            else:
                upd_indices += [i]

        p[:,to_mod] += [p_unif]*len(to_mod)
    print(np.sum(p))


    # Confirmation bias
    #conf_bias = np.zeros((1, A.shape[0]))
    #conf_bias[:,from_color] = conf_bias[:,from_color] + conf_bias_param*conf_bias[:,from_color]

    #iterations = 15
    p_new = p @ A
    p_new = p_new/np.sum(p_new)
    d = [1/(i+1) for i in p_new[0]]
    diagonal = diags(d, offsets=0)
    D = (diagonal @ A.transpose()).transpose()
    norm = [1/i[0] if i!=0 else 1 for i in np.array(D.sum(axis=1))[:]]
    D = diags(norm, offsets=0) @ D
    #rrrr = [np.mean(p_new[:,to_color][p_new[:,to_color]!=0])]
    #mmmm = [np.mean(p_new[:,to_color])]
    ssss = [np.sum(p_new[:,to_color])]
    same = [np.sum(p_new[:,from_color])]
    #same_avg = [np.mean(p_new[:,from_color])]
    #same_avg_ = [np.mean(p_new[:,from_color][p_new[:,from_color]!=0])]
    #nnnn = [np.sum(p_new[:,to_color])/(np.sum(p_new[:,to_color])+np.sum(p_new[:,from_color]))]
    
    return ssss, same#mmmm#rrrr, mmmm, ssss, same, same_avg, same_avg_, nnnn

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.colors import LinearSegmentedColormap


# AsItIs

In [ ]:
sample_iter = 1
for topic in ['abortion','cannabis','guns','evolution', 'racism','lgbt']:
    with open(MATRIX_FOLDER  + topic + '_' + 'uniform' + '_' + '_adj_matrix.p', 'rb') as f_pickle_info:
        A = pickle.load(f_pickle_info)
    with open(MATRIX_FOLDER  + topic + '_' + 'uniform' + '_' + 'node_id.p', 'rb') as f_pickle_info:
        node_id = pickle.load(f_pickle_info)
    with open(MATRIX_FOLDER  + topic + '_' + 'uniform' + '_' + 'node_color.p', 'rb') as f_pickle_info:
        node_color = pickle.load(f_pickle_info)

    id_node = {j:i for i,j in node_id.items()}
    # Get nodes
    red = []
    blue = []
    green = []
    for n in node_id:
        if node_color[n] == 'R':
            red.append(n)
        elif node_color[n] == 'B':
            blue.append(n)
        elif node_color[n] == 'G':
            green.append(n)
    
    print('NUM {}:{}'.format(dictionary_name_cat[topic]['R'], len(red)))
    print('NUM {}:{}'.format(dictionary_name_cat[topic]['B'], len(blue)))

    random_samples = []

    if len(red) < len(blue):
        for rand_op in range(sample_iter): 
            random_samples += [blue]#[random.sample(blue, len(red))]

    elif len(red) > len(blue):
        for rand_op in range(sample_iter): 
            random_samples += [red]#[random.sample(red, len(blue))]


    dictionary_sequences = {}

    for alpha in [0]:#, 0.2, 0.5, 0.8, 1]:  
        print(alpha)
        dictionary_sequences[alpha] = {}


        for kind in ['uniform', 'position', 'clicks']:
            print(kind)

            #kind = 'uniform'
            with open(MATRIX_FOLDER  + topic + '_' + kind + '_' + '_adj_matrix.p', 'rb') as f_pickle_info:
                A = pickle.load(f_pickle_info)
            with open(MATRIX_FOLDER  + topic + '_' + kind + '_' + 'node_id.p', 'rb') as f_pickle_info:
                node_id = pickle.load(f_pickle_info)
            with open(MATRIX_FOLDER  + topic + '_' + kind + '_' + 'node_color.p', 'rb') as f_pickle_info:
                node_color = pickle.load(f_pickle_info)

            id_node = {j:i for i,j in node_id.items()}
            # Get nodes
            red = []
            blue = []
            green = []
            for n in node_id:
                if node_color[n] == 'R':
                    red.append(node_id[n])
                elif node_color[n] == 'B':
                    blue.append(node_id[n])
                elif node_color[n] == 'G':
                    green.append(node_id[n])

            dictionary_sequences[alpha][kind] = {}
            #dictionary_sequences[alpha][kind]['red'] = {}
            #dictionary_sequences[alpha][kind]['blue'] = {}
            #dictionary_sequences[alpha][kind]['red_red'] = {}
            #dictionary_sequences[alpha][kind]['blue_blue'] = {}


            print(alpha)
            red_repetitions = []
            blue_repetitions = []
            red_red_repetitions = []
            blue_blue_repetitions = []
            if len(red) < len(blue):
                for rand_op in range(sample_iter):
                    #print(rand_op)
                    blue_random = [node_id[node] for node in random_samples[rand_op]]#random.sample(blue, len(red))
                    sim_rb, sim_rr = average_opposite_color(list(red), list(blue_random), A, alpha=alpha)
                    sim_br, sim_bb = average_opposite_color(list(blue_random), list(red), A, alpha=alpha)
                    red_repetitions += [sim_rb]
                    blue_repetitions += [sim_br]
                    red_red_repetitions += [sim_rr]
                    blue_blue_repetitions += [sim_bb]
                    #print(red_repetitions,blue_repetitions)
            elif len(red) > len(blue):
                for rand_op in range(sample_iter):
                    print(rand_op)
                    red_random = [node_id[node] for node in random_samples[rand_op]]#random.sample(red, len(blue))
                    sim_rb, sim_rr = average_opposite_color(list(red_random), list(blue), A, alpha=alpha)
                    sim_br, sim_bb = average_opposite_color(list(blue), list(red_random), A, alpha=alpha)

                    red_repetitions += [sim_rb]
                    blue_repetitions += [sim_br]
                    red_red_repetitions += [sim_rr]
                    blue_blue_repetitions += [sim_bb]


            dictionary_sequences[alpha][kind]['red'] = (np.mean(np.array(red_repetitions), axis=0), np.std(np.array(red_repetitions), axis=0), st.t.interval(0.90, len(np.array(red_repetitions))-1, loc=np.mean(np.array(red_repetitions)), scale=st.sem(np.array(red_repetitions))))#uniform_from_red np.std(np.array(red_repetitions), axis=0)
            dictionary_sequences[alpha][kind]['blue'] = (np.mean(np.array(blue_repetitions), axis=0),np.std(np.array(blue_repetitions), axis=0), st.t.interval(0.90, len(np.array(blue_repetitions))-1, loc=np.mean(np.array(blue_repetitions)), scale=st.sem(np.array(blue_repetitions))))#uniform_from_blue


            dictionary_sequences[alpha][kind]['red_red'] = (np.mean(np.array(red_red_repetitions), axis=0),np.std(np.array(red_red_repetitions), axis=0), st.t.interval(0.90, len(np.array(red_red_repetitions))-1, loc=np.mean(np.array(red_red_repetitions)), scale=st.sem(np.array(red_red_repetitions))))#uniform_from_red
            dictionary_sequences[alpha][kind]['blue_blue'] = (np.mean(np.array(blue_blue_repetitions), axis=0),np.std(np.array(blue_blue_repetitions), axis=0), st.t.interval(0.90, len(np.array(blue_blue_repetitions))-1, loc=np.mean(np.array(blue_blue_repetitions)), scale=st.sem(np.array(blue_blue_repetitions))))#uniform_from_blue

    with open('data/models/' + topic + '_exdin' + '_sequences_KDD_intervals_AII.p', 'wb') as f_pickle_info:
        pickle.dump(dictionary_sequences, f_pickle_info)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.colors import LinearSegmentedColormap

list_values = []
for k in ['uniform','clicks']:#,'position'
    list_values += [round(dictionary_sequences[0][k]['red_red'][0][0]*100,2)]
    list_values += [round(dictionary_sequences[0][k]['red'][0][0]*100,2)]
    list_values += [round(dictionary_sequences[0][k]['blue'][0][0]*100,2)]
    list_values += [round(dictionary_sequences[0][k]['blue_blue'][0][0]*100,2)]


fig, ax = plt.subplots(3, 6,  figsize=(24,6))
plt.rcParams['axes.edgecolor']='#333F4B'
plt.rcParams['axes.linewidth']=0.5


#cmap = plt.cm.get_cmap('Blues', 20)#cm.rainbow
norm = colors.Normalize(vmin=min(list_values), vmax=max(list_values))
cmap_p = LinearSegmentedColormap.from_list('mycmap', ['#D08BAF', '#8B2E5F', '#450025'])

f2rgb = cm.ScalarMappable(norm=norm, cmap=cmap_p)#cmap=cm.get_cmap('Oranges', 10))

def f2hex(f2rgb, f):
    rgb = f2rgb.to_rgba(f)[:3]
    return '#%02x%02x%02x' % tuple([int(255*fc) for fc in rgb])


#norm = matplotlib.colors.Normalize(vmin=min(list_values), vmax=max(list_values))

kinds = ['uniform', 'clicks']#
r = [0.5]#,100]
r2 = [1]
b = [0.5]#
b2 = [1]
#temp2=list(df_not_acc['B'])
#city=['Pro-life', 'Pro-choice','Gun control', 'Gun rights',  
#      'Creazionism', 'Evolutionary biology','LGBT discrimination',
#      'LGBT rights','Church and State','Freedom on religion','White supremacy','Anti-racism',]

y_pos= [0, 0, ]#np.array(list(range(len(temp))))
y_pos2 = [1,1]#np.array(list(range(len(temp)))) + .45

topics = ['abortion','cannabis','guns','evolution', 'racism','lgbt']

list_sequences = []
for t in topics:
    with open('data/models/' + t + '_exdin' + '_sequences_KDD_intervals_AII.p', 'rb') as f_pickle_info:
            dictionary_sequences = pickle.load(f_pickle_info)
            list_sequences += [dictionary_sequences]

#for topic in topics:
for i in range(3):
    for j in np.arange(6):#range(3):


        ax[i,j].text(0.76, 0.22, str(round(list_sequences[j][0][kinds[i]]['red_red'][0][0]*100,2)),
                          fontsize=16, ha='center', color='white')
        
        #ax[i,j].text(0.76, 0.15, '[{},{}]'.format(round(list_sequences[j][0][kinds[i]]['red_red'][2][0][0]*100,2),round(list_sequences[j][0][kinds[i]]['red_red'][2][1][0]*100,2)),
        #             fontsize=13, ha='center', color='white')
        
        
        ax[i,j].text(0.24, 0.72, str(round(list_sequences[j][0][kinds[i]]['blue_blue'][0][0]*100,2)),
                          fontsize=16, ha='center', color='white')
        #ax[i,j].text(0.24, 0.15, '[{},{}]'.format(round(list_sequences[j][0][kinds[i]]['blue'][2][0][0]*100,2),round(list_sequences[j][0][kinds[i]]['blue'][2][1][0]*100,2)),
        #             fontsize=13, ha='center', color='white')
        
        
        ax[i,j].text(0.76, 0.72, str(round(list_sequences[j][0][kinds[i]]['red'][0][0]*100,2)),
                          fontsize=16, ha='center', color='white')
        #ax[i,j].text(0.76, 0.65, '[{},{}]'.format(round(list_sequences[j][0][kinds[i]]['red'][2][0][0]*100,2),round(list_sequences[j][0][kinds[i]]['red'][2][1][0]*100,2)),
        #             fontsize=13, ha='center', color='white')
        
        ax[i,j].text(0.24, 0.22, str(round(list_sequences[j][0][kinds[i]]['blue'][0][0]*100,2)),
                          fontsize=16, ha='center', color='white')
        
        #ax[i,j].text(0.24, 0.65, '[{},{}]'.format(round(list_sequences[j][0][kinds[i]]['blue_blue'][2][0][0]*100,2),round(list_sequences[j][0][kinds[i]]['blue_blue'][2][1][0]*100,2)),
        #             fontsize=13, ha='center', color='white')
        


        ax[i,j].bar([0.25], r2,color=f2hex(f2rgb, round(list_sequences[j][0][kinds[i]]['blue_blue'][0][0]*100,2)), alpha=1, width=0.5)
        ax[i,j].bar([0.25], r,color=f2hex(f2rgb, round(list_sequences[j][0][kinds[i]]['red'][0][0]*100,2)), alpha=1, width=0.5)#'#6b5b95'

        ax[i,j].bar([.76], b2,color=f2hex(f2rgb, round(list_sequences[j][0][kinds[i]]['blue'][0][0]*100,2)), alpha=1, width=0.5)
        ax[i,j].bar([0.76], b,color=f2hex(f2rgb, round(list_sequences[j][0][kinds[i]]['red_red'][0][0]*100,2)), alpha=1, width=0.5)##feb236
    #plt.barh(y_pos2, temp2,color='orange', alpha=.5, height=.4)
    #plt.barh(np.array(y_pos)+.4, temp,color=['blue','orange'], alpha=.5)


        ax[i,j].axvline(x=0.5, ymax=1, color='white', linestyle='-')
        ax[i,j].axhline(y=0.5, xmin=0, xmax=1, color='white', linestyle='-')

        ax[i,j].set_yticks([0.25,.75])
        ax[i,j].set_yticklabels([dictionary_name_cat[topics[j]]['R'], dictionary_name_cat[topics[j]]['B']], fontsize=14)
        if i==1:
            ax[i,j].set_xticks([0.25,.75]) 
            ax[i,j].set_xticklabels([dictionary_name_cat[topics[j]]['B'], dictionary_name_cat[topics[j]]['R']], fontsize=14, rotation=90)
        if i!=1:
            ax[i,j].set_xticks([]) 
            ax[i,j].set_xticklabels([])
        #ax[i].title('Proca')
        #ax[i].xlabel('')
        if j==0:
            ax[i,j].set_ylabel(kinds[i].capitalize(), fontsize=20)
        ax[i,j].set_ylim(0, 1)
        ax[i,j].set_xlim(0, 1)

    
#sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
#sm.set_array([])  # only needed for matplotlib < 3.1
#fig.colorbar(sm)
#plt.xticks([0.25,.75],[dictionary_name_cat[topic]['R'], dictionary_name_cat[topic]['B']], fontsize=12, rotation=40)
#plt.yticks([0.25,.75],[dictionary_name_cat[topic]['R'], dictionary_name_cat[topic]['B']], fontsize=12)
# removing axes from the figure
plt.gca().spines['right'].set_visible(True)
plt.gca().spines['top'].set_visible(True)
plt.gca().spines['bottom'].set_visible(True)
#plt.gca().spines['left'].set_smart_bounds(True)

fig.tight_layout(pad=.3)
#plt.savefig('plots/' +'MATRIX_exdin_KDD.pdf', bbox_inches = 'tight')
plt.show()

# Bootstrapping

In [ ]:
sample_iter = 100
for topic in ['abortion','cannabis','guns','evolution', 'racism','lgbt']:
    with open(MATRIX_FOLDER  + topic + '_' + 'uniform' + '_' + '_adj_matrix.p', 'rb') as f_pickle_info:
        A = pickle.load(f_pickle_info)
    with open(MATRIX_FOLDER  + topic + '_' + 'uniform' + '_' + 'node_id.p', 'rb') as f_pickle_info:
        node_id = pickle.load(f_pickle_info)
    with open(MATRIX_FOLDER  + topic + '_' + 'uniform' + '_' + 'node_color.p', 'rb') as f_pickle_info:
        node_color = pickle.load(f_pickle_info)

    id_node = {j:i for i,j in node_id.items()}
    # Get nodes
    red = []
    blue = []
    green = []
    for n in node_id:
        if node_color[n] == 'R':
            red.append(n)
        elif node_color[n] == 'B':
            blue.append(n)
        elif node_color[n] == 'G':
            green.append(n)
    
    print('NUM {}:{}'.format(dictionary_name_cat[topic]['R'], len(red)))
    print('NUM {}:{}'.format(dictionary_name_cat[topic]['B'], len(blue)))

    random_samples_red = []
    random_samples_blue = []
    
    min_len = min(len(red), len(blue))

    
    for rand_op in range(sample_iter): 
        random_samples_blue += [np.random.choice(blue, min_len)]#[blue]#[random.sample(blue, len(red))]
    
    for rand_op in range(sample_iter): 
        random_samples_red += [random.sample(red, min_len)]#[red]#[random.sample(red, len(blue))]

    dictionary_sequences = {}

    for alpha in [0]:#, 0.2, 0.5, 0.8, 1]:  
        print(alpha)
        dictionary_sequences[alpha] = {}


        for kind in ['uniform', 'position', 'clicks']:
            print(kind)

            #kind = 'uniform'
            with open(MATRIX_FOLDER  + topic + '_' + kind + '_' + '_adj_matrix.p', 'rb') as f_pickle_info:
                A = pickle.load(f_pickle_info)
            with open(MATRIX_FOLDER  + topic + '_' + kind + '_' + 'node_id.p', 'rb') as f_pickle_info:
                node_id = pickle.load(f_pickle_info)
            with open(MATRIX_FOLDER  + topic + '_' + kind + '_' + 'node_color.p', 'rb') as f_pickle_info:
                node_color = pickle.load(f_pickle_info)

            id_node = {j:i for i,j in node_id.items()}
            # Get nodes
            red = []
            blue = []
            green = []
            for n in node_id:
                if node_color[n] == 'R':
                    red.append(node_id[n])
                elif node_color[n] == 'B':
                    blue.append(node_id[n])
                elif node_color[n] == 'G':
                    green.append(node_id[n])

            dictionary_sequences[alpha][kind] = {}
            #dictionary_sequences[alpha][kind]['red'] = {}
            #dictionary_sequences[alpha][kind]['blue'] = {}
            #dictionary_sequences[alpha][kind]['red_red'] = {}
            #dictionary_sequences[alpha][kind]['blue_blue'] = {}


            print(alpha)
            red_repetitions = []
            blue_repetitions = []
            red_red_repetitions = []
            blue_blue_repetitions = []
            #if len(red) < len(blue):
            for rand_op in range(sample_iter):
                #print(rand_op)
                red_random = [node_id[node] for node in random_samples_red[rand_op]]
                blue_random = [node_id[node] for node in random_samples_blue[rand_op]]#random.sample(blue, len(red))
                sim_rb, sim_rr = average_opposite_color_bootstrapping(list(red_random), list(blue_random), A, alpha=alpha)
                sim_br, sim_bb = average_opposite_color_bootstrapping(list(blue_random), list(red_random), A, alpha=alpha)
                red_repetitions += [sim_rb]
                blue_repetitions += [sim_br]
                red_red_repetitions += [sim_rr]
                blue_blue_repetitions += [sim_bb]
                    #print(red_repetitions,blue_repetitions)
            #elif len(red) > len(blue):
            #    for rand_op in range(sample_iter):
            #        print(rand_op)
            #        red_random = [node_id[node] for node in random_samples[rand_op]]#random.sample(red, len(blue))
            #        sim_rb, sim_rr = average_opposite_color(list(red_random), list(blue), A, alpha=alpha)
            #        sim_br, sim_bb = average_opposite_color(list(blue), list(red_random), A, alpha=alpha)

            #        red_repetitions += [sim_rb]
            #        blue_repetitions += [sim_br]
            #        red_red_repetitions += [sim_rr]
            #        blue_blue_repetitions += [sim_bb]


            dictionary_sequences[alpha][kind]['red'] = (np.mean(np.array(red_repetitions), axis=0), np.std(np.array(red_repetitions), axis=0), st.t.interval(0.90, len(np.array(red_repetitions))-1, loc=np.mean(np.array(red_repetitions)), scale=st.sem(np.array(red_repetitions))))#uniform_from_red np.std(np.array(red_repetitions), axis=0)
            dictionary_sequences[alpha][kind]['blue'] = (np.mean(np.array(blue_repetitions), axis=0),np.std(np.array(blue_repetitions), axis=0), st.t.interval(0.90, len(np.array(blue_repetitions))-1, loc=np.mean(np.array(blue_repetitions)), scale=st.sem(np.array(blue_repetitions))))#uniform_from_blue


            dictionary_sequences[alpha][kind]['red_red'] = (np.mean(np.array(red_red_repetitions), axis=0),np.std(np.array(red_red_repetitions), axis=0), st.t.interval(0.90, len(np.array(red_red_repetitions))-1, loc=np.mean(np.array(red_red_repetitions)), scale=st.sem(np.array(red_red_repetitions))))#uniform_from_red
            dictionary_sequences[alpha][kind]['blue_blue'] = (np.mean(np.array(blue_blue_repetitions), axis=0),np.std(np.array(blue_blue_repetitions), axis=0), st.t.interval(0.90, len(np.array(blue_blue_repetitions))-1, loc=np.mean(np.array(blue_blue_repetitions)), scale=st.sem(np.array(blue_blue_repetitions))))#uniform_from_blue

    with open('data/models/' + topic + '_exdin' + '_sequences_KDD_intervals_boot.p', 'wb') as f_pickle_info:
        pickle.dump(dictionary_sequences, f_pickle_info)

In [ ]:
with open('data/models/' + topic + '_exdin' + '_sequences_KDD_intervals_boot.p', 'rb') as f_pickle_info:
    dictionary_sequences = dict(pickle.load(f_pickle_info))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.colors import LinearSegmentedColormap

list_values = []
for k in ['uniform','position']:#'clicks']:#,
    list_values += [round(dictionary_sequences[0][k]['red_red'][0][0]*100,2)]
    list_values += [round(dictionary_sequences[0][k]['red'][0][0]*100,2)]
    list_values += [round(dictionary_sequences[0][k]['blue'][0][0]*100,2)]
    list_values += [round(dictionary_sequences[0][k]['blue_blue'][0][0]*100,2)]


fig, ax = plt.subplots(2, 6,  figsize=(24,6))
plt.rcParams['axes.edgecolor']='#333F4B'
plt.rcParams['axes.linewidth']=0.5


#cmap = plt.cm.get_cmap('Blues', 20)#cm.rainbow
norm = colors.Normalize(vmin=min(list_values), vmax=max(list_values))
cmap_p = LinearSegmentedColormap.from_list('mycmap', ['#D08BAF', '#8B2E5F', '#450025'])

f2rgb = cm.ScalarMappable(norm=norm, cmap=cmap_p)#cmap=cm.get_cmap('Oranges', 10))

def f2hex(f2rgb, f):
    rgb = f2rgb.to_rgba(f)[:3]
    return '#%02x%02x%02x' % tuple([int(255*fc) for fc in rgb])


#norm = matplotlib.colors.Normalize(vmin=min(list_values), vmax=max(list_values))

kinds = ['uniform',  'position']#'clicks'],
r = [0.5]#,100]
r2 = [1]
b = [0.5]#
b2 = [1]
#temp2=list(df_not_acc['B'])
#city=['Pro-life', 'Pro-choice','Gun control', 'Gun rights',  
#      'Creazionism', 'Evolutionary biology','LGBT discrimination',
#      'LGBT rights','Church and State','Freedom on religion','White supremacy','Anti-racism',]

y_pos= [0, 0, ]#np.array(list(range(len(temp))))
y_pos2 = [1,1]#np.array(list(range(len(temp)))) + .45

topics = ['abortion','cannabis','guns','evolution', 'racism','lgbt']

list_sequences = []
for t in topics:
    with open('data/models/' + t + '_exdin' + '_sequences_KDD_intervals_boot.p', 'rb') as f_pickle_info:
            dictionary_sequences = pickle.load(f_pickle_info)
            list_sequences += [dictionary_sequences]

#for topic in topics:
for i in range(2):
    for j in np.arange(6):#range(3):


        ax[i,j].text(0.76, 0.27, str(round(list_sequences[j][0][kinds[i]]['red_red'][0][0]*100,2)),
                          fontsize=16, ha='center', color='white')
        
        ax[i,j].text(0.76, 0.15, '[{},{}]'.format(round(list_sequences[j][0][kinds[i]]['red_red'][2][0][0]*100,2),round(list_sequences[j][0][kinds[i]]['red_red'][2][1][0]*100,2)),
                     fontsize=13, ha='center', color='white')
        
        
        ax[i,j].text(0.24, 0.77, str(round(list_sequences[j][0][kinds[i]]['blue_blue'][0][0]*100,2)),
                          fontsize=16, ha='center', color='white')
        ax[i,j].text(0.24, 0.15, '[{},{}]'.format(round(list_sequences[j][0][kinds[i]]['blue'][2][0][0]*100,2),round(list_sequences[j][0][kinds[i]]['blue'][2][1][0]*100,2)),
                     fontsize=13, ha='center', color='white')
        
        
        ax[i,j].text(0.76, 0.77, str(round(list_sequences[j][0][kinds[i]]['red'][0][0]*100,2)),
                          fontsize=16, ha='center', color='white')
        ax[i,j].text(0.76, 0.65, '[{},{}]'.format(round(list_sequences[j][0][kinds[i]]['red'][2][0][0]*100,2),round(list_sequences[j][0][kinds[i]]['red'][2][1][0]*100,2)),
                     fontsize=13, ha='center', color='white')
        
        ax[i,j].text(0.24, 0.27, str(round(list_sequences[j][0][kinds[i]]['blue'][0][0]*100,2)),
                          fontsize=16, ha='center', color='white')
        
        ax[i,j].text(0.24, 0.65, '[{},{}]'.format(round(list_sequences[j][0][kinds[i]]['blue_blue'][2][0][0]*100,2),round(list_sequences[j][0][kinds[i]]['blue_blue'][2][1][0]*100,2)),
                     fontsize=13, ha='center', color='white')
        


        ax[i,j].bar([0.25], r2,color=f2hex(f2rgb, round(list_sequences[j][0][kinds[i]]['blue_blue'][0][0]*100,2)), alpha=1, width=0.5)
        ax[i,j].bar([0.25], r,color=f2hex(f2rgb, round(list_sequences[j][0][kinds[i]]['red'][0][0]*100,2)), alpha=1, width=0.5)#'#6b5b95'

        ax[i,j].bar([.76], b2,color=f2hex(f2rgb, round(list_sequences[j][0][kinds[i]]['blue'][0][0]*100,2)), alpha=1, width=0.5)
        ax[i,j].bar([0.76], b,color=f2hex(f2rgb, round(list_sequences[j][0][kinds[i]]['red_red'][0][0]*100,2)), alpha=1, width=0.5)##feb236
    #plt.barh(y_pos2, temp2,color='orange', alpha=.5, height=.4)
    #plt.barh(np.array(y_pos)+.4, temp,color=['blue','orange'], alpha=.5)


        ax[i,j].axvline(x=0.5, ymax=1, color='white', linestyle='-')
        ax[i,j].axhline(y=0.5, xmin=0, xmax=1, color='white', linestyle='-')

        ax[i,j].set_yticks([0.25,.75])
        ax[i,j].set_yticklabels([dictionary_name_cat[topics[j]]['R'], dictionary_name_cat[topics[j]]['B']], fontsize=14)
        if i==1:
            ax[i,j].set_xticks([0.25,.75]) 
            ax[i,j].set_xticklabels([dictionary_name_cat[topics[j]]['B'], dictionary_name_cat[topics[j]]['R']], fontsize=14, rotation=90)
        if i!=1:
            ax[i,j].set_xticks([]) 
            ax[i,j].set_xticklabels([])
        #ax[i].title('Proca')
        #ax[i].xlabel('')
        if j==0:
            ax[i,j].set_ylabel(kinds[i].capitalize(), fontsize=20)
        ax[i,j].set_ylim(0, 1)
        ax[i,j].set_xlim(0, 1)

    
#sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
#sm.set_array([])  # only needed for matplotlib < 3.1
#fig.colorbar(sm)
#plt.xticks([0.25,.75],[dictionary_name_cat[topic]['R'], dictionary_name_cat[topic]['B']], fontsize=12, rotation=40)
#plt.yticks([0.25,.75],[dictionary_name_cat[topic]['R'], dictionary_name_cat[topic]['B']], fontsize=12)
# removing axes from the figure
plt.gca().spines['right'].set_visible(True)
plt.gca().spines['top'].set_visible(True)
plt.gca().spines['bottom'].set_visible(True)
#plt.gca().spines['left'].set_smart_bounds(True)

fig.tight_layout(pad=.3)
plt.savefig('plots/' +'MATRIX_exdin_KDD_bootstrap.pdf', bbox_inches = 'tight')
plt.show()

# Same color vs Opposite color

In [ ]:
def average_opposite_color_multiplt_boot(from_color, to_color, A, alpha=0.2, iterations=10):
    
    #initial_red = np.array([1 for n in from_color])
    #norm_initial_red = initial_red/np.sum(initial_red)
    #p = np.zeros((1, A.shape[0]))
    #p[:,from_color] = norm_initial_red
    p_unif = 1/len(from_color)

    initial_red = np.array([1 for n in from_color])
    norm_initial_red = initial_red/np.sum(initial_red)
    p = np.zeros((1, A.shape[0]))
    indices = from_color

    to_mod = []
    upd_indices = []
    for i in indices:
        if i not in to_mod:
            to_mod += [i]
        else:
            upd_indices += [i]

    initial_red = np.array(list(set(from_color)))
    p[:,initial_red] = [p_unif]*len(initial_red)

    #print(np.sum(p))

    while len(to_mod) > 0:
        indices = upd_indices
        to_mod = []
        upd_indices = []
        #print(indices)
        for i in indices:
            if i not in to_mod:
                to_mod += [i]
            else:
                upd_indices += [i]

        p[:,to_mod] += [p_unif]*len(to_mod)

    # Confirmation bias
    #conf_bias = np.zeros((1, A.shape[0]))
    #conf_bias[:,from_color] = conf_bias[:,from_color] + conf_bias_param*conf_bias[:,from_color]

    #iterations = 15
    p_new = p @ A
    p_new = p_new/np.sum(p_new)
    d = [1/(i+1) for i in p_new[0]]
    diagonal = diags(d, offsets=0)
    D = (diagonal @ A.transpose()).transpose()
    norm = [1/i[0] if i!=0 else 1 for i in np.array(D.sum(axis=1))[:]]
    D = diags(norm, offsets=0) @ D
    rrrr = [np.mean(p_new[:,to_color][p_new[:,to_color]!=0])]
    mmmm = [np.mean(p_new[:,to_color])]
    ssss = [np.sum(p_new[:,to_color])]
    same = [np.sum(p_new[:,from_color])]
    same_avg = [np.mean(p_new[:,from_color])]
    same_avg_ = [np.mean(p_new[:,from_color][p_new[:,from_color]!=0])]
    nnnn = [np.sum(p_new[:,to_color])/(np.sum(p_new[:,to_color])+np.sum(p_new[:,from_color]))]
    
    for it in range(iterations):    
        #conf_bias[:,from_color] = conf_bias[:,from_color] + conf_bias_param*(conf_bias_param*conf_bias[:,from_color])
        p_tmp = (1-alpha)*(p_new @ D) + alpha*(p @ D) #+  bias*conf_bias
        p_tmp = p_tmp/np.sum(p_tmp)
        p_new = p_tmp

        d = [1/(i+1) for i in p_new[0]]
        diagonal = diags(d, offsets=0)

        D = (diagonal @ D.transpose()).transpose()
        norm = [1/i[0] if i!=0 else 1 for i in np.array(D.sum(axis=1))[:]]
        D = diags(norm, offsets=0) @ D
        
        
        #rrrr += [np.mean(p_new[:,to_color][p_new[:,to_color]!=0])]
        #mmmm += [np.mean(p_new[:,to_color])]
        ssss += [np.sum(p_new[:,to_color])]
        same += [np.sum(p_new[:,from_color])]
        #same_avg += [np.mean(p_new[:,from_color])]
        #same_avg_ += [np.mean(p_new[:,from_color][p_new[:,from_color]!=0])]
        #nnnn += [np.sum(p_new[:,to_color])/(np.sum(p_new[:,to_color])+np.sum(p_new[:,from_color]))]
        
    return ssss, same#mmmm#rrrr, mmmm, ssss, same, same_avg, same_avg_, nnnn


In [ ]:
#a = np.array(dictionary_sequences['clicks']['blue'][0][0])
fig, ax = plt.subplots(4, 6,  figsize=(16,7))

topics = ['abortion', 'cannabis', 'guns', 'evolution','racism', 'lgbt']

for i in range(3):
    for j in range(6):
        topic = topics[j]
        with open('data/models/' + topic + '_' + 'sequences_bootstrapping_KDD.p', 'rb') as f_pickle_info:
            dictionary_sequences = dict(pickle.load(f_pickle_info))
        
        if i == 2:
            for alpha,c in [(0,'#02D40F'), (0.2,'#2A4ED2')]:#, (1,'#02D40F')]:
                for kind,m in [ ('uniform','-o'), ('clicks','-x')]:#, ('clicks','-^')#('position','-s'), #, ('clicks','--o'), 'position', 'clicks']:#'occurrences', 
                
                    b = np.array(dictionary_sequences[alpha][kind]['blue_to_red'][0])[:15]
                    a = np.array(dictionary_sequences[alpha][kind]['red_to_blue'][0])[:15]
                    std_b = np.array(dictionary_sequences[alpha][kind]['blue_to_red'][1])[:15]/np.sqrt(len(b))
                    std_a = np.array(dictionary_sequences[alpha][kind]['red_to_blue'][1])[:15]/np.sqrt(len(a))
                    max_array = np.array(list(map(lambda x: max(x[0],x[1]), zip(a, b))))
                    min_array = np.array(list(map(lambda x: min(x[0],x[1]), zip(a, b))))

                    mins = [np.argmin([i,j]) for i,j in  zip(a, b)]
                    maxs = [np.argmax([i,j]) for i,j in  zip(a, b)]
                    zip_se = list(zip(std_a,std_b))
                    sMin = np.array([zip_se[i][m] for i,m in enumerate(mins)])
                    sMax = np.array([zip_se[i][m] for i,m in enumerate(maxs)])
                    se_tot = min_array/max_array * np.sqrt((sMin/min_array)**2 + (sMax/max_array)**2)*100

                    ratio = min_array/max_array*100

                    ax[i,j].plot(ratio, m, color=c, label=kind + ' alpha=' + str(alpha),
                             alpha=.7, linewidth=.8, markersize=5,fillstyle='none')
                    ax[i,j].fill_between(range(len(b)), y1=ratio-se_tot, color=c, y2=ratio+se_tot, alpha=.1)
        elif i == 1:
            for alpha,c in [(0,'#02D40F'), (0.2,'#2A4ED2')]:#, (1,'#02D40F')]:
                for kind,m in [ ('uniform','-o'), ('clicks','-x')]:#, ('clicks','-^')# ,('position','-s') #,('position', '-.'),('clicks','--o'), 'position', 'clicks']:#'occurrences', 
                    
                    ax[i,j].plot(dictionary_sequences[alpha][kind]['blue_out_in'][0], m, color=c, label=kind + ' alpha=' + str(alpha), 
                                 alpha=.7, linewidth=.8, fillstyle='none')
                    
                    l = dictionary_sequences[alpha][kind]['blue_out_in'][0]
                    std = dictionary_sequences[alpha][kind]['blue_out_in'][1]

                    #plt.plot(np.array(dictionary_sequences[kind]['blue'][alpha][1])[:15], m, color='red', label=kind + ' alpha=' + str(alpha), alpha=.5)
                    ax[i,j].fill_between(range(len(l)), y1=np.array(l)-np.array(std), color=c, y2=np.array(l)+np.array(std), alpha=.1)
        
        elif i == 0:
            for alpha,c in [(0,'#02D40F'), (0.2,'#2A4ED2')]:#, (1,'#02D40F')]:
                for kind,m in [('uniform','-o'), ('clicks','-x')]:#, ('clicks','-^')#('position','-s'),#,('position', '-.'),('clicks','--o'), 'position', 'clicks']:#'occurrences', 
                    
                    ax[i,j].plot(dictionary_sequences[alpha][kind]['red_out_in'][0], m, color=c, label=kind + ' alpha=' + str(alpha), 
                                 alpha=.7, linewidth=.8, fillstyle='none')
                    l = dictionary_sequences[alpha][kind]['red_out_in'][0]
                    std = dictionary_sequences[alpha][kind]['red_out_in'][1]
                    #plt.plot(np.array(dictionary_sequences[kind]['blue'][alpha][1])[:15], m, color='red', label=kind + ' alpha=' + str(alpha), alpha=.5)
                    ax[i,j].fill_between(range(len(l)), y1=np.array(l)-np.array(std), color=c, y2=np.array(l)+np.array(std), alpha=.1)
            
            
        b = (dictionary_sequences[alpha][kind]['blue_to_red'][0])[:15]
            
        ax[i,j].set_xticks(list(np.arange(0,len(b),2)))
        ax[i,j].set_xticklabels(np.arange(1,len(b)+1,2), fontsize=10, rotation=0)
        #ax[i].set_yticks([0.25,.75]) 
        #ax[i].set_xticklabels([0.25,.75],[dictionary_name_cat[topic]['R'], dictionary_name_cat[topic]['B']], fontsize=12, rotation=90)
        if i == 0:
            ax[i,j].set_title(topics[j].capitalize(), size=13)
        #ax[i].xlabel('')
        #if i==2:
        #    ax[i,j].set_xlabel('Number of clicks', fontsize=15, rotation=0)
        #ax[j].set_ylim(0, 1)
        #ax[i].set_xlim(0, 1)

        ax[i,j].grid(True,axis='y', alpha=.3, linestyle='--')
        ax[i,j].spines['right'].set_visible(False)
        ax[i,j].spines['top'].set_visible(False)

#handles, labels = ax[2,1].get_legend_handles_labels()

#ax[2,2].legend(handles, labels, loc='upper center',
#           bbox_to_anchor=(0.5, -.69), ncol=3, fontsize=12)
#plt.legend(loc='center', bbox_to_anchor=(1, -0.4),
#          shadow=False, ncol=3)#fancybox=True

ax[0,0].set_ylabel(r'$\frac{e^{\ell}_{P\rightarrow \bar{P}}}{e_{P\rightarrow P}}$', fontsize=20)
ax[1,0].set_ylabel(r'$\frac{e^{\ell}_{\bar{P}\rightarrow P}}{e_{\bar{P} \rightarrow \bar{P}}}$', fontsize=20)
ax[2,0].set_ylabel(r'$\epsilon^{\ell}$', fontsize=15)
#plt.xlabel('Number of clicks', fontsize=15)
#plt.xticks(range(len(b)), range(1,len(b)+1))


fig.tight_layout()
fig.text(0.45, 0.23, 'Number of Clicks', va='center', rotation='horizontal', fontsize=15)

for kk in range(6):
    ax[3,kk].spines['bottom'].set_visible(False)
    ax[3,kk].spines['left'].set_visible(False)
    ax[3,kk].spines['right'].set_visible(False)
    ax[3,kk].spines['top'].set_visible(False)
    ax[3,kk].spines['top'].set_visible(False)
    ax[3,kk].set_xticks([])
    ax[3,kk].set_xticklabels([])
    ax[3,kk].set_yticks([])
    ax[3,kk].set_yticklabels([])
    
    
    
handles, labels = ax[2,1].get_legend_handles_labels()
fig.legend(handles, labels, bbox_to_anchor=(.5, -0.05), 
            loc='lower center', borderaxespad=5, ncol=2)#(handles, labels, loc='lower center')

#fig3.savefig('plots/' + 'temporal_graph_mexdin.pdf', bbox_inches = 'tight')


plt.savefig('all_uniform_start_Mexdin_KDD.pdf', bbox_inches = 'tight')
plt.show()